# SBN Reader - Locosys SiRF Binary

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

from datetime import datetime, timezone

import numpy as np

import unittest

from sirf_reader import SirfReader

## Main Class

In [2]:
class SbnReader(SirfReader):
    '''SBN file - Locosys SiRF Binary'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)
       

    def readData(self):
        '''Read geodetic navigation data into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            # A small handful of SBN files from the GT-11 ended with a single byte 0x0a
            if self.bufferPtr == fileSize - 1 and self.buffer[self.bufferPtr] == 0x0a:
                self.readBytes(1)
            else:
                messageId, message = self.readMessage()

                # Geodetic Navigation Data
                if messageId == 41:
                    # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                    leanBuffer += message
                    self.numRecords += 1

        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        if len(leanBuffer) % self.numRecords != 0:
            raise ValueError('File appears to have varying message lengths')
        messageLen = len(leanBuffer) // self.numRecords

        dtype = self.getDatatype(messageLen)

        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def getDatatype(self, messageLen):
        '''Get the datatype of the SBN messages'''

        # Standard 95-byte SiRF geodetic navigation data
        dtype = [
            ('message_id', 'u1'),             # 41 = geodetic navigation data
            ('nav_valid', '>u2'),             # Typically 0
            ('nav_type', '>u2'),              # See SiRF binary protocol
            ('extended_week_no', '>u2'),
            ('time_of_week', '>u4'),
            ('utc_year', '>u2'),
            ('utc_month', 'u1'),
            ('utc_day', 'u1'),
            ('utc_hour', 'u1'),
            ('utc_minute', 'u1'),
            ('utc_millisecs', '>u2'),
            ('sv_ids', '>u4'),
            ('latitude', '>i4'),
            ('longitude', '>i4'),
            ('altitude_ellipsoid', '>i4'),
            ('altitude_msl', '>i4'),
            ('map_datum', 'u1'),             # e.g. 21 = WGS-84
            ('sog', '>u2'),
            ('cog', '>u2'),
            ('magnetic_variation', '>i2'),   # Not implemented
            ('climb_rate', '>i2'),
            ('heading_rate', '>i2'),         # Not implemented - SiRFDRive only
            ('ehpe', '>u4'),
            ('evpe', '>u4'),
            ('ete', '>u4'),                  # Not implemented - SiRFDRive only
            ('ehve', '>u2'),                 # Not implemented - SiRFDRive only
            ('clock_bias', '>i4'),
            ('clock_bias_error', '>u4'),     # Not implemented - SiRFDRive only
            ('clock_drift', '>i4'),
            ('clock_drift_error', '>u4'),    # Not implemented - SiRFDRive only
            ('distance', '>u4'),             # Not implemented - SiRFDRive only
            ('distance_error', '>u2'),       # Not implemented - SiRFDRive only
            ('heading_error', '>u2'),        # Not implemented - SiRFDRive only
            ('sv_count', 'u1'),
            ('hdop', 'u1'),
            ('additional_mode_info', 'u1')   # Typically 0
        ]

        # Locosys message extensions (GT-11)
        if messageLen >= 95:
            dtype.extend([
                ('unfiltered_sog', '>u2'),
                ('unfiltered_cog', '>u2')
            ])

        # Locosys message extensions (GT-31 onwards)
        if messageLen >= 97:
            dtype.extend([
                ('sdop', 'u1'),
                ('vsdop', 'u1')
            ])

        # Forward compatibility / future-proofing in case SBN messages exceed 97 bytes in length
        for i in range(messageLen - 97):
            dtype.extend([
                ('unknown_{:02}'.format(i), 'u1')
            ])

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert date/time to regular timestamp (seconds)
        vfunc = np.vectorize(getDateTime)
        self.data['timestamp'] = vfunc( self.rawData['utc_year'], self.rawData['utc_month'], self.rawData['utc_day'],
                                        self.rawData['utc_hour'], self.rawData['utc_minute'], self.rawData['utc_millisecs'])

        # Consume universal SiRF binary fields
        super().consumeRawData()
        
        # Convert estimated positional errors from cm to m
        self.data['ehpe'] = self.rawData['ehpe'] / 100
        self.data['evpe'] = self.rawData['evpe'] / 100

        # Convert unfiltered SOG from cm/s to m/s
        self.data['usog'] = self.rawData['unfiltered_sog'] / 100

        # Convert unfiltered COG from integer to decimal
        self.data['ucog'] = self.rawData['unfiltered_cog'] / 100

        # Convert SDOP from cm/s to m/s - original SBN format was without SDOP
        if 'sdop' in self.rawData.dtype.names:
            self.data['sdop'] = self.rawData['sdop'] / 100

        # Convert VSDOP from cm/s to m/s - original SBN format was without VSDOP
        if 'vsdop' in self.rawData.dtype.names:
            self.data['vsdop'] = self.rawData['vsdop'] / 100
            

def getDateTime(year, month, day, hour, minute, millisecs):
    '''Decode date/time'''

    return datetime(year, month, day, hour, minute, millisecs // 1000, millisecs % 1000, tzinfo=timezone.utc).timestamp()

## Unit Tests

In [3]:
class TestHeader(unittest.TestCase):
    '''Class to test header was correctly loaded'''

    def testUsername(self):
        '''Test the username is as expected'''

        self.assertEqual(sbnReader.header['username'], 'GEORG30MICHA')


    def testSerial(self):
        '''Test the serial is as expected'''

        self.assertEqual(sbnReader.header['serial'], 932000175)


    def testFrequency(self):
        '''Test the frequency is as expected'''

        self.assertEqual(sbnReader.header['frequency'], 1)


    def testFirmware(self):
        '''Test the firmware is as expected'''

        self.assertEqual(sbnReader.header['firmware'], 'V1.4(B0803T)')

In [4]:
class TestData(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbnReader.numRecords, 4161)

        for fieldName in sbnReader.data:
            self.assertEqual(sbnReader.data[fieldName].size, 4161)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbnReader.data['hdop'].min(), 0.8)
        self.assertEqual(sbnReader.data['hdop'].max(), 2.0)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbnReader.data['sat'].min(), 4)
        self.assertEqual(sbnReader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbnReader.data['timestamp'].min(), 1649672182)
        self.assertEqual(sbnReader.data['timestamp'].max(), 1649678792)


    def testSvIds(self):
        '''Test the satellite IDs are as expected'''

        self.assertEqual(sbnReader.data['sv_ids'].min(), 302123008)
        self.assertEqual(sbnReader.data['sv_ids'].max(), 3542222850)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbnReader.data['lat'].min(), 50.571016)
        self.assertEqual(sbnReader.data['lat'].max(), 50.5833319)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbnReader.data['lon'].min(), -2.4620455)
        self.assertEqual(sbnReader.data['lon'].max(), -2.4563038)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(sbnReader.data['ele'].min(), -3.02)
        self.assertEqual(sbnReader.data['ele'].max(), 11.93)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbnReader.data['sog'].min(), 0.13)
        self.assertEqual(sbnReader.data['sog'].max(), 16.83)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbnReader.data['cog'].min(), 0.01)
        self.assertEqual(sbnReader.data['cog'].max(), 359.92)


    def testRoc(self):
        '''Test the rate of climb is as expected'''

        self.assertEqual(sbnReader.data['roc'].min(), -0.56)
        self.assertEqual(sbnReader.data['roc'].max(), 0.35)


    def testEhpe(self):
        '''Test the estimated horizontal positional error is as expected'''

        self.assertEqual(sbnReader.data['ehpe'].min(), 0.57)
        self.assertEqual(sbnReader.data['ehpe'].max(), 7.57)


    def testEhve(self):
        '''Test the estimated vertical positional error is as expected'''

        self.assertEqual(sbnReader.data['evpe'].min(), 0.8)
        self.assertEqual(sbnReader.data['evpe'].max(), 4.54)


    def testUsog(self):
        '''Test the unfiltered speed over ground is as expected'''

        self.assertEqual(sbnReader.data['usog'].min(), 0.13)
        self.assertEqual(sbnReader.data['usog'].max(), 16.83)


    def testUcog(self):
        '''Test the unfiltered course over ground is as expected'''

        self.assertEqual(sbnReader.data['ucog'].min(), 0.01)
        self.assertEqual(sbnReader.data['ucog'].max(), 359.92)


    def testUsogDiff(self):
        '''Test the difference in unfiltered speed over ground is as expected'''

        self.assertEqual(np.abs(np.around(sbnReader.data['sog'] - sbnReader.data['usog'], 2)).max(), 0.19)


    def testUcogDiff(self):
        '''Test the difference in unfiltered course over ground is as expected'''

        self.assertEqual(np.abs(np.around(sbnReader.data['cog'] - sbnReader.data['ucog'], 2)).max(), 0.01)


    def testSdop(self):
        '''Test the speed dilution of precision is as expected'''

        self.assertEqual(sbnReader.data['sdop'].min(), 0.09)
        self.assertEqual(sbnReader.data['sdop'].max(), 2.1)


    def testVsdop(self):
        '''Test the vertical speed dilution of precision is as expected'''

        self.assertEqual(sbnReader.data['vsdop'].min(), 0.1)
        self.assertEqual(sbnReader.data['vsdop'].max(), 0.39)

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.SBN')
    sbnReader = SbnReader(filename)

    pc1 = time.perf_counter()
    sbnReader.load()
    pc2 = time.perf_counter()

    print("\nSBN file loaded in in %0.2f seconds" % (pc2 - pc1))


SBN file loaded in in 0.02 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.......................
----------------------------------------------------------------------
Ran 23 tests in 0.013s

OK
